In [ ]:
import gurobipy as gp
from gurobipy import GRB
from automata.fa.dfa import DFA
from pydot import Dot, Edge, Node
import itertools
from time import time
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def data_preprocess(m):
    #Function to preprocess data and extract necessary information

    #Lst constains list of sequences, and FL contains a list of lists, each containing sequences separated by commas
    with open(f"C:/Users/bchan/Desktop/TUD/gurobi/Master_Roy/train_10{m}.txt", "r") as my_file:
        Lst = [line.strip() for line in my_file.readlines()]
    
    FL = [l.split(",") for l in Lst]
        
    uniq_list=set(Lst)
    alphabet = set(item for string in uniq_list for item in string.split(",") if item != ',')
    
    Pref_S = set()
    for string in uniq_list:
        prefixes = string.split(',')
        for i in range(1, len(prefixes) + 1):
            prefix = ','.join(prefixes[:i])
            Pref_S.add(prefix)
    Pref_S.add('')
    
    return alphabet, Pref_S, Lst, FL


def train(n, m, lower):
    #Function to train a model and return a DFA
    #n denotes no of states, m denotes the ALFRED goal, lower denotes the lower bounds

    alphabet, Pref_S, Lst, FL = data_preprocess(m)
    states = {str(f'q{i}') for i in range(n)}
    start_state = 'q0'

    env=gp.Env(empty=True)
    env.setParam('OutputFlag', 0)
    env.start()

    # Creating a new model
    msba = gp.Model("DFA_SBA", env=env)

    msba.setParam('Seed', 10)

    t0 = time()
    #DECISION VARIABLES
    delta = msba.addVars(states, alphabet, states, vtype=gp.GRB.BINARY, name='delta')
    x = msba.addVars(Pref_S, states, vtype=gp.GRB.BINARY, name='x')
    f = msba.addVars(states, vtype=gp.GRB.BINARY, name='f')
    alpha = msba.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'alpha')
    LB = msba.addVar(lb=lower,ub=lower,vtype=gp.GRB.CONTINUOUS, name='LB')
    
    #OBJECTIVE
    msba.setObjective(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states ), gp.GRB.MINIMIZE)
                                   
    #Self-loop: 
    #sum(delta[state0,symbol,state1] for state0 in states for symbol in alphabet for state1 in states if state0 != state1)
    #Parallel-edge: 
    '''msba.setObjective(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states ) \
                +  sum(delta[state1,symbol,state2]*eps for state1 in states for symbol in alphabet for state2 in states if state1 != state2) \
                   , gp.GRB.MINIMIZE)'''

    #AUTOMATA CONSTRAINTS
    #Constraint1
    for state0 in states:
        for symbol in alphabet:
            msba.addConstr(sum(delta[state0,symbol,state1] for state1 in states)==1, name=f'delta[{state0},{symbol}]')
    
    #Constraint2
    for word in Pref_S:
        msba.addConstr(sum(x[word,state1] for state1 in states)==1, name=f'x[{word}]')

    #Constraint3
    msba.addConstr(x['',start_state]==1, name='initial_state')

    #Constraint4
    for state0, word, symbol, state1 in itertools.product(states, Pref_S, alphabet, states):
        if (word + ',' + symbol) in Pref_S:
            msba.addConstr(x[word,state0] + delta[state0, symbol, state1] -1 <= x[word + ',' + symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')

        if word == '' and symbol in Pref_S:
            msba.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')

    #BOUND CONSTRAINTS
    for i, word in enumerate(Lst):
        for state1 in states:
            msba.addConstr(alpha[i, state1] >= x[word,state1] + f[state1] -1, name=f'bound_1[{state1},{i}]')
            msba.addConstr(alpha[i, state1] <= x[word,state1], name=f'bound_2[{state1},{i}]')
            msba.addConstr(alpha[i, state1] <= f[state1], name=f'bound_3[{state1},{i}]')        
         
    msba.addConstr(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states )/len(Lst) >= LB, name=f'lowerBound')
    
    #Write the model
    msba.write(rf'C:\Users\bchan\Desktop\TUD\Thesis\model_SB_{n}.lp')

    msba.optimize()
    #print('Obj: %g' % msba.ObjVal)

    t1 = time()
    print("Run time", (t1-t0), "seconds")

    if msba.status == 1:
        status = 'LOADED'
        print(f'DFAmodel_{n}: {status}')
            
    elif msba.status == 2:
        print(f'DFAmodel_{n} OPTIMAL')
        status='OPTIMAL'
        transitions = msba.getAttr('X', delta)
        t_values = [(s1,a,s2) for s1 in states for s2 in states for a in alphabet if round(transitions[s1, a, s2],0) == 1]
        f_s = msba.getAttr('X', f)
        #print(f_s)
        final_state = {s1 for s1 in states if round(f_s[s1],0) == 1}

        transition_dict = create_transition_dict(states, alphabet, t_values)
        
        dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states=final_state)
        accepted = 0
        rejected = 0
        for w in FL:
            if dfa1.accepts_input(w):
                accepted += 1             
            else:
                rejected += 1        

        #create_diagram(rf'C:\Users\bchan\Desktop\TUD\Thesis\diagram_SB_{n}.png', states, start_state,final_state, transition_dict)
        return dfa1    
    
    elif msba.status == 3:
        status = 'INFEASIBLE'
        print(f'DFAmodel_{n}: {status}')
    else:
        print('status unknown, DEBUG!!')    


def create_transition_dict(states, alphabet, t_values):
    # Function to create a transition dictionary from transition values

    transition_dict = {}

    for state in states:
        transition_dict[state] = {}
        for symbol in alphabet:
            transition_dict[state][symbol] = None

    for trans in t_values:
        current_state, symbol, next_state = trans
        transition_dict[current_state][symbol] = next_state

    return transition_dict

def create_diagram(path, states, start_state, final_state, transition_dict):
    # Function to create a visualization diagram of the DFA

    graph = Dot(graph_type='digraph', rankdir='LR')
    nodes = {}
    for state in states:
        if state == start_state:
            # color start state with green
            if state in final_state:
                initial_state_node = Node(
                    state,
                    style='filled',
                    peripheries=2,
                    fillcolor='#66cc33')
            else:
                initial_state_node = Node(
                    state, style='filled', fillcolor='#66cc33')
            nodes[state] = initial_state_node
            graph.add_node(initial_state_node)
        else:
            if state in final_state:
                state_node = Node(state, peripheries=2)
            else:
                state_node = Node(state)
            nodes[state] = state_node
            graph.add_node(state_node)
    # adding edges
    for from_state, lookup in transition_dict.items():
        for to_label, to_state in lookup.items():
            graph.add_edge(Edge(
                nodes[from_state],
                nodes[to_state],
                label=to_label
            ))
    if path:
        graph.write_png(path)
    return graph

def test(m, dfa1, correct_label):
    # Function to test the DFA model on a test dataset and evaluate performance

    with open(f"C:/Users/bchan/Desktop/TUD/gurobi/Master_Roy/test_10{m}.txt", "r") as my_file:
        lines = [line.strip() for line in my_file.readlines()]        
    
    #Lst1, FL1 for test dataset is same as Lst and FL for train dataset
    Lst1, FL1, G = [], [], []

    for line in lines:
        Lst_line,g = tuple(line.rstrip().split(";"))
        Lst1.append(Lst_line)
        FL1.append(Lst_line.split(','))
        if int(g)==correct_label:
            G.append(0)
        else:
            G.append(1)

    accepted = 0
    rejected = 0
    Predicted_labels=[]
    for w in FL1:
        if dfa1.accepts_input(w):
            Predicted_labels.append(1)
            accepted += 1             
        else:
            Predicted_labels.append(0)
            rejected += 1 

    accuracy = accuracy_score(G, Predicted_labels)
    print(f'Accuracy:{round(accuracy,2)}')
    f1 = f1_score(G, Predicted_labels, average='binary', pos_label=1)
    print(f'F1_score:{round(f1,2)}\n')

#Main Loop for Training and Testing
#n denotes no of states and g denotes the ALFRED goal
#lower denotes the lower bounds
for n in range(2,3):
    g=6
    lower=0.11
    #eps=0.1
    dfa1= train(n, g, lower)
    test(m=g, dfa1=dfa1, correct_label=g)
